In [1]:

from graphilp.imports import graph_formats
from graphilp.imports import networkx as imp_nx
import helpers
import networkx as nx
from gurobipy import *
import xlsxwriter
import os
from datetime import datetime
from pcstHeuristicModule import *
import heuristicConfig as conf
import numpy as np
%load_ext autoreload
%autoreload 2
foundTime = datetime.now()

In [24]:
filePath = os.path.join(os.getcwd(), 'testInstances', 'tooBig', 'G101.stp')
G, terminals = graph_formats.stp_to_networkx(filePath)
#G, terminals, root = graph_formats.rooted_pcst_to_networkx(filePath)
G2 = G.to_undirected()
G = nx.DiGraph(G)


prizes = []
costs = []
edges=[]
#root = 13
# Giving each node a random prize if its a terminal. If its a steiner Node, set its prize to 0.
for node in G2.nodes():
    if node in terminals:
        thisPrize = random.randint(10000, 100000)
        prizes.append(thisPrize)
        G.nodes[node]['prize'] = thisPrize
    else:
        prizes.append(0)
        G.nodes[node]['prize'] = 0

# Set each edges capacity
for edge in G.edges():
    G[edge[0]][edge[1]]['capacity'] = conf.capcity

# create reverse edge for every edge in the graph
for edge in G.edges(data=True):
    if ((edge[1], edge[0]) in G.edges()):
        continue
    else:
        G.add_edge(edge[1], edge[0], weight = edge[2]['weight'], capacity = edge[2]['capacity'], prize = edge[2]['prize'])
        
sum_of_prizes = sum(prizes)

# Generating input data for PCST Fast
for edge in G2.edges(data=True):
    costs.append(edge[2]['weight'])
    edges.append([edge[0] - 1,edge[1] - 1])
    
prizesNP = np.array(prizes, dtype = float)
costsNP = np.array(costs, dtype = float)
edgesNP = np.array(edges, dtype = int)

In [25]:
# from pcstHeuristicModule_loops import *
from pcstHeuristicModule import *

# Setting a node as a root. 
Graph = imp_nx.read(G)
#root = 69

# Compute the Linear Relaxation of the Model
solutionEdges, solutionNodes, result = computeLP(G, Graph, terminals)

# Change this depending on whether you want to classical solution with for loops or the one 
# with pandas DataFrames. The Pandas DataFrames doesn't work yet. Thus, only use the second method
# for test purposes only!
startTime = datetime.now()
pd_edges, pd_nodes = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
                                          solutionNodes = solutionNodes, root = root)
result, timeElapsed = networkImprovement(G, pd_edges, pd_nodes, terminals, root)
# z, y, f = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
#                               solutionNodes = solutionNodes, root = root)
# result, timeElapsed = networkImprovement(G, z, y, f, terminals, root)
print(datetime.now() - startTime)
with open('outputBIG.txt', 'a') as f:
    f.write(str(result) + ", " + str(timeElapsed))

5212728
construction again 3.0688085556030273
construction again 3.19956374168396
construction again 3.4610002040863037
construction again 151.09040522575378
construction again 3.2039995193481445
construction again 3.1650335788726807
construction again 3.218548536300659
construction again 207.92753624916077
construction again 3.5512068271636963
construction again 4.255403518676758
construction again 3.02500057220459
construction again 200.06236696243286
construction again 4.140103101730347
construction again 4.252999544143677
construction again 4.325353384017944
construction again 199.4514901638031
construction again 4.11165714263916
construction again 3.9330015182495117
construction again 3.884000539779663
construction again 3.0450332164764404
construction again 4.986018896102905
construction again 4.066920757293701
construction again 4.063403606414795
construction again 4.072999954223633
construction again 4.004597425460815
construction again 3.668609380722046
construction again 3.84

Ivana Ljubic Heuristic on Cologne Instances - Vectorized

Showdown between PCST_Fast and My Heuristic

In [1]:
from pcstHeuristicModule import *
%load_ext autoreload
%autoreload 2

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks_PCSTHeuristicss.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'Terminals', bold)
    worksheet.write('E1', 'Upper bound', bold)
    worksheet.write('F1', 'Time2Sol', bold)
    cell_format = workbook.add_format()
    cell_format.set_num_format('hh:mm:ss')
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            if j == 5:
                worksheet.write(i + 1, j, solution[i][j], cell_format)
            else:
                worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark():
    solutions = []
    toTest = ['i103M2.stp']
    i = 0
    print(pd.__version__)
    prizes = []
    costs = []
    edges = []
    
    basepath = os.path.join('testInstances', 'PCST')
    for fname in os.listdir(basepath):
        if fname not in toTest:
            continue
        path = os.path.join(os.getcwd(), basepath, fname)
        if os.path.isdir(path):
            # skip directories
            continue
            
        with open(path) as f:
            G, terminals, root = graph_formats.rooted_pcst_to_networkx(path)
            G = nx.DiGraph(G)
            for edge in G.edges():
                G[edge[0]][edge[1]]['capacity'] = 15

            # create reverse edge for every edge in the graph
            for edge in G.edges(data=True):
                if ((edge[1], edge[0]) in G.edges()):
                    continue
                else:
                    G.add_edge(edge[1], edge[0], weight = edge[2]['weight'], capacity = edge[2]['capacity'])

            Graph = imp_nx.read(G)
            root = 1
            solutionEdges, solutionNodes, result = computeLP(G, Graph, terminals)
            
            pd_edges, pd_nodes = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
                                          solutionNodes = solutionNodes, root = root)
            
            result, timeElapsed = networkImprovement(G, pd_edges, pd_nodes, terminals, root)
            with open('output1.txt', 'a') as f:
                f.write(str(result) + ", " + str(timeElapsed))
            solutions.append((fname, len(G.edges()), len(G.nodes()), len(terminals), result, timeElapsed))

    __write_solution(solutions)         

if __name__ == '__main__':
    runBenchmark()
    

1.2.1


NameError: name 'graph_formats' is not defined

In [3]:
#from pcst_fast import pcst_fast
import numpy as np

fname = 'i105M3.stp'
i = 0

prizes = []
costs = []
edges = []

basepath = os.path.join('testInstances', 'PCST')
path = os.path.join(os.getcwd(), basepath, fname)
prizes = []
weights = []
costs = []

with open(path) as f:
    print(fname)
    G, terminals, root = graph_formats.rooted_pcst_to_networkx(path)
    G = G.to_undirected() 
    
    for edge in G.edges():
        G[edge[0]][edge[1]]['capacity'] = 8
    for node in G.nodes():
        prizes.append(G.nodes[node]['prize'])

    # create reverse edge for every edge in the graph
    for edge in G.edges(data=True):
        if ((edge[1], edge[0]) in G.edges()):
            continue
        else:
            G.add_edge(edge[1], edge[0], weight = edge[2]['weight'], capacity = edge[2]['capacity'])
    sum_of_prizes = sum(prizes)

    for edge in G.edges(data=True):
        costs.append(edge[2]['weight'])
        edges.append([edge[0] - 1,edge[1] - 1])
    
    Graph = imp_nx.read(G)
    prizesNP = np.array(prizes, dtype = float)
    costsNP = np.array(costs, dtype = float)
    edgesNP = np.array(edges, dtype = int)
    vertices, edges = pcst_fast(edgesNP, prizesNP, costsNP, -1, 1, 'gw', 1)

    helpers.computeMinimizationResultPCSTFast(sum_of_prizes, G, edges, G.edges)



i105M3.stp


NameError: name 'pcst_fast' is not defined

In [ ]:
# Get the PCST Fast solution
vertices, edges = pcst_fast(edgesNP, prizesNP, costsNP, root, 1, 'gw', 1)
# Compute a comparable solution objective
computeMinimizationResultPCSTFast(sum_of_prizes, G, edges, G2.edges)

Test of Loops Module

In [25]:
from pcstHeuristicModule_loops import *
def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks_PCSTHeuristicss.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'Terminals', bold)
    worksheet.write('E1', 'Upper bound', bold)
    worksheet.write('F1', 'Time2Sol', bold)
    cell_format = workbook.add_format()
    cell_format.set_num_format('hh:mm:ss')
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            if j == 5:
                worksheet.write(i + 1, j, solution[i][j], cell_format)
            else:
                worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark():
    solutions = []
    toTest = ['i101M2.stp']
    
    basepath = os.path.join('testInstances', 'PCST')
    for fname in os.listdir(basepath):
        if fname not in toTest:
            continue
        path = os.path.join(os.getcwd(), basepath, fname)
        if os.path.isdir(path):
            # skip directories
            continue
            
        with open(path) as f:
            G, terminals, root = graph_formats.rooted_pcst_to_networkx(path)
            G = nx.DiGraph(G)
            for edge in G.edges():
                G[edge[0]][edge[1]]['capacity'] = conf.copperCap

            # create reverse edge for every edge in the graph
            for edge in G.edges(data=True):
                if ((edge[1], edge[0]) in G.edges()):
                    continue
                else:
                    G.add_edge(edge[1], edge[0], weight = edge[2]['weight'], capacity = edge[2]['capacity'])

            Graph = imp_nx.read(G)
            root = 1
            solutionEdges, solutionNodes, result = computeLP(G, Graph, terminals)
            z, y, f = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
                                          solutionNodes = solutionNodes, root = root)
            result, timeElapsed = networkImprovement(G, z, y, f, terminals, root)
            print(result, timeElapsed)
            solutions.append((fname, len(G.edges()), len(G.nodes()), len(terminals), result, timeElapsed))

    __write_solution(solutions)         

if __name__ == '__main__':
    runBenchmark()
    

383401.34763399995
162956.16627299995
1
162956.16627299995
2
162956.16627299995
3
162956.16627299995
4
382962.681315
163394.83259200002
0
163394.83259200002
1
163394.83259200002
2
163394.83259200002
3
163394.83259200002
4
163394.83259200002
5
378051.681315
168305.83259200002
0
168305.83259200002
1
168305.83259200002
2
168305.83259200002
3
168305.83259200002
4
168305.83259200002
5
168305.83259200002
6
377434.681315
168922.83259200002
0
168922.83259200002
1
168922.83259200002
2
168922.83259200002
3
168922.83259200002
4
168922.83259200002
5
168922.83259200002
6
168922.83259200002
7
168922.83259200002
8
370080.804755
176276.70915200008
0
176276.70915200008
1
176276.70915200008
2
176276.70915200008
3
176276.70915200008
4
176276.70915200008
5
176276.70915200008
6
176276.70915200008
7
176276.70915200008
8
176276.70915200008
9
368788.742533
177568.77137400006
0
177568.77137400006
1
177568.77137400006
2
177568.77137400006
3
177568.77137400006
4
177568.77137400006
5
177568.77137400006
6
177568.7